In [ ]:
#Keras support
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121 as DenseNet
from keras.applications.resnet50 import ResNet50 as ResNet
from keras.layers import Input, Dense, Flatten
from keras.models import Model, Sequential

#Plotting
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt

from pandas import read_csv

import numpy as np

from common import constants

In [ ]:
label_df = read_csv(constants.PROCESSED_DATASET_MAPPINGS['labels'])
n_classes = len(set(label_df["Id"]))

print("Number of classes: {}".format(n_classes))

In [ ]:
inputs = Input(shape = (224, 224, 3))
X = DenseNet(weights = 'imagenet')(inputs)
X = Dense(n_classes, activation = 'softmax')(X)

model = Model(inputs = [inputs], outputs = [X])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

In [ ]:
input_shape = (224, 224, 3)

base_model = ResNet(include_top=False, weights='imagenet', input_shape=input_shape)

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(300, activation="softmax"))

for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

In [ ]:
input_shape_no_channel = (224, 224)
batch_size = 64

datagen = ImageDataGenerator(
                    rescale=1./255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    validation_split = 0.2)

train_gen = datagen.flow_from_dataframe(
                    label_df,
                    constants.PROCESSED_DATASET_MAPPINGS['train'],
                    x_col = 'Image',
                    y_col = 'Id',
                    target_size=input_shape_no_channel,
                    batch_size=batch_size,
                    class_mode='categorical',
                    subset = 'training')

val_gen = datagen.flow_from_dataframe(
                    label_df,
                    constants.PROCESSED_DATASET_MAPPINGS['train'],
                    x_col = 'Image',
                    y_col = 'Id',
                    target_size=input_shape_no_channel,
                    batch_size=batch_size,
                    class_mode='categorical',
                    subset='validation')

In [ ]:
X, Y = train_gen.next()
n_images = 2 # X.shape[0]


output = model.predict(X)

In [ ]:
n_images = X.shape[0]

"""
nn = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
nbrs = nn.kneighbors(X[0], n_neighbors=5, return_distance=True)
print(nbrs.shape)
"""

for img_row in range(n_images):
    rows = []
    for img_col in range(n_images):
        rows.append(np.linalg.norm(output[img_row] - output[img_col]))
    
    img_id = np.argmax(rows)
    print((img_row, img_id), np.amax(rows), (np.nonzero(Y[img_row]), np.nonzero(Y[img_id])))
    
figure, axes = plt.subplots(2, 2)
axes[0, 0].imshow(X[4])
axes[0, 1].imshow(X[14])
axes[1, 0].imshow(X[41])
axes[1, 1].imshow(X[42])



In [ ]:
import pandas as pd
import numpy as np

In [1]:
#Keras imports
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, BatchNormalization, Activation, Concatenate
from keras.models import Model

#Feature model
from resnet_feature_model import resnet_feature_model as feature_model

#Data processing
import numpy as np
import pandas as pd

#Load and save objects to disk
from pandas import read_csv

#Allow reproducible results
from numpy.random import seed as np_seed
from tensorflow import set_random_seed as tf_seed

#Constants
from common import constants

def siamese_network_model(input_shape, feature_dims):
    anchor_input = Input(shape = input_shape, name = 'Anchor')
    sample_input = Input(shape = input_shape, name = 'Sample')

    anchor_features = feature_model(input_shape, feature_dims)(anchor_input)
    sample_features = feature_model(input_shape, feature_dims)(sample_input)

    X = Concatenate()([anchor_features, sample_features])
    X = Dense(16, activation = 'linear')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Dense(4, activation = 'linear')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Dense(1, activation = 'sigmoid')(X)

    siamese_network = Model(inputs = [anchor_input, sample_input], outputs = [X], name = 'Siamese Model')
    siamese_network.summary()

    return siamese_network

def make_datagen(batch_size, validation_split = None):
    idg_kwargs = dict(
                    rescale = 1./255, shear_range = 0.2,
                    rotation_range=10, width_shift_range=0.2,
                    height_shift_range=0.2, zoom_range = 0.2, horizontal_flip= True)

    datagen = None

    if validation_split:
        datagen = ImageDataGenerator(validation_split = validation_split, **idg_kwargs)
    else:
        datagen = ImageDataGenerator(**idg_kwargs)

    return datagen

def make_generators(datagen, source, train_tuples_df, x_col, y_col, batch_size, validation_split = None):
    input_shape_nc = (224, 224)
    val_gen = None

    flow_kwargs = dict(
                    x_col = x_col, y_col = y_col, target_size = input_shape_nc,
                    batch_size = batch_size)

    if validation_split:
        val_gen = datagen.flow_from_dataframe(train_tuples_df, source, subset = 'validation', **flow_kwargs)

    train_gen = datagen.flow_from_dataframe(train_tuples_df, source, subset = 'training', **flow_kwargs)

    return train_gen, val_gen

if __name__ == "__main__":
    #Fix randomness
    seed = 3
    np_seed(3)
    tf_seed(3)
    df_image_col = constants.IMAGE_HEADER_NAME
    df_class_col = constants.LABEL_HEADER_NAME
    input_shape = constants.INPUT_SHAPE
    feature_dims = constants.FEATURE_VECTOR_DIMS
    train_set_loc = constants.PROCESSED_DATASET_MAPPINGS['train']
    anchor_field = constants.TRAIN_TUPLE_HEADERS[0]
    sample_field = constants.TRAIN_TUPLE_HEADERS[1]
    similar_field = constants.TRAIN_TUPLE_HEADERS[3]

    batch_size = 32

    train_tuples_df = read_csv(constants.PROCESSED_DATASET_MAPPINGS['train_tuples'])
    datagen = make_datagen(batch_size)
    anchor_train_gen, _ = make_generators(datagen, train_set_loc, train_tuples_df, anchor_field, similar_field, batch_size)
    sample_train_gen, _ = make_generators(datagen, train_set_loc, train_tuples_df, sample_field, similar_field, batch_size)

    batch_anchor = anchor_train_gen.next()
    batch_sample = sample_train_gen.next()

    print(batch_sample[0].shape)

    #model = siamese_network_model(input_shape, feature_dims)

Using TensorFlow backend.
C:\Users\nares\Anaconda3\lib\site-packages\keras_preprocessing\image.py:2059: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.df[x_col] = self.df[x_col].astype(str)


Found 1024 images belonging to 1 classes.
Found 1024 images belonging to 2 classes.
(32, 224, 224, 3)


In [2]:
print(batch_sample[1].shape)
print(batch_anchor[1].shape)

(32, 2)
(32, 1)


In [ ]:
from pandas import read_csv
from common import constants

train_tuples_loc = constants.PROCESSED_DATASET_MAPPINGS['train_tuples']
train_tuples_df = read_csv(train_tuples_loc)